In [1]:
import random
import time
import sys
from collections import Counter
from statistics import mean, median
import numpy as np
import grpc

import simulation_service_pb2 as v1
import simulation_service_pb2_grpc as v1GRPC

# Before running this...
You'll need to have the training server running locally on your machine. Find more info about that [here](https://olamai.gitbook.io/olamai/training-models/installing-the-environment)

# Connecting to the training server

In [2]:
channel = grpc.insecure_channel('localhost:9090')
stub = v1GRPC.SimulationServiceStub(channel)
metadata = [('auth-secret', 'MOCK-SECRET')]
api="v1"

# Helper Functions #

In [3]:
def randomAction():
    rand = random.randint(1, 3)
    if rand == 1:
        return "MOVE"
    if rand == 2:
        return "CONSUME"
    if rand == 3:
        return "WAIT"
    
def randomDirection():
    rand = random.randint(1, 4)
    if rand == 1:
        return "UP"
    if rand == 2:
        return "DOWN"
    if rand == 3:
        return "LEFT"
    if rand == 4:
        return "RIGHT"

# Commands For Training #

In [4]:
stub.ResetWorld(v1.ResetWorldRequest(), metadata=metadata)

In [5]:
createAgentResp = stub.CreateAgent(v1.CreateAgentRequest(api="v1", modelName="My Model", x=0, y=0), metadata=metadata)
agentId = createAgentResp.id

In [6]:
action = v1.Action(id="MOVE", direction="UP")
actionRes = stub.ExecuteAgentAction(v1.ExecuteAgentActionRequest(id=agentId, action=action))
actionRes.wasActionSuccessful

False

In [7]:
obsvRes = stub.GetAgentObservation(v1.GetAgentObservationRequest(id=agentId))
obsv = obsvRes.observation
cellList = list(obsv.cells)
cellList.insert(4, 'A')
cellMatrix = np.array(cellList).reshape(3,3)
# Full observation response
print(obsv)
# Formatted as what the agent sees
print(cellMatrix)

id: 389
cells: "EMPTY"
cells: "FOOD"
cells: "EMPTY"
cells: "EMPTY"
cells: "EMPTY"
cells: "EMPTY"
cells: "EMPTY"
cells: "EMPTY"
alive: true
energy: 98
health: 100

[['EMPTY' 'FOOD' 'EMPTY']
 ['EMPTY' 'A' 'EMPTY']
 ['EMPTY' 'EMPTY' 'EMPTY']]


# Training Example #

In [8]:
initial_games = 50
goal_steps = 500
score_requirement = 32
training_data = []
scores = []
accepted_scores = []
# Metadata values that need to be sent in order for the calls to be succesful
api="v1"
# Run through some iterations
for episode in range(initial_games):
    # Reset the world
    stub.ResetWorld(v1.ResetWorldRequest(), metadata=metadata)
    # Reset game memory
    game_memory = []
    # Spawn a new agent
    spawnAgentRes = stub.CreateAgent(v1.CreateAgentRequest(api="v1", modelName="My Model", x=0, y=0), metadata=metadata)
    agentId = spawnAgentRes.id 
    # Score (essentially the age of the agent)
    score = 0
    # Perform actions and observations
    for t in range(goal_steps):
        obsvResp = stub.GetAgentObservation(v1.GetAgentObservationRequest(api=api, id=agentId), metadata=metadata)
        if obsvResp.observation.alive == False:
            break
        action = randomAction()
        direction = randomDirection()
        action = v1.Action(id=action, direction=direction)
        actionRes = stub.ExecuteAgentAction(v1.ExecuteAgentActionRequest(api=api, id=agentId, action=action), metadata=metadata)
        score += 1
        game_memory.append( [list(obsvResp.observation.cells), action] )
        # time.sleep(.20)
    
    if score >= score_requirement:
        accepted_scores.append(score)
        for data in game_memory:
            training_data.append(data)
    
    scores.append(score)

training_data_save = np.array(training_data)
np.save('saved_training_data.npy', training_data_save)

print('Average accepted score: ', mean(accepted_scores))
print('Median accepted score : ', median(accepted_scores))
print(Counter(accepted_scores))
        

Average accepted score:  64
Median accepted score :  65.0
Counter({60: 22, 65: 18, 70: 9, 80: 1})
